In [96]:
# Personal Library
import project_baseball as project
# Pybaseball
import pybaseball as pb
# Packages
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

In [97]:
# Import Data 
mlb2020 = pd.read_csv('data/2020mlb.csv')
print('mlb 2020 data shape: {0}'.format(mlb2020.shape))

mlb 2020 data shape: (279660, 90)


# Feature Selection

---
The feature columns selected will be to help determine the what type of pitch is thrown. 

These features may also help in determining the type of pitch that will be thrown in sequence. I will also gather data on the players who are batting as much as possible.

In [98]:
# These columns are considered irrelevant as to having way too many nulls
mlb2020.drop(columns=['spin_dir', 'spin_rate_deprecated',
       'break_angle_deprecated', 'break_length_deprecated','tfs_deprecated', 'tfs_zulu_deprecated',
       'umpire', 'sv_id','pitcher.1'], inplace=True)

In [99]:
mlb2020.head(2)

,index,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,fielder_2,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment
0,0,FF,2020-10-27,96.7,1.58,5.99,Julio Urias,642715.0,628711.0,strikeout,called_strike,4.0,Willy Adames called out on strikes.,W,R,L,LAD,TB,S,2.0,NaN,0.0,2.0,2020.0,0.2,1.6,-0.53,2.29,NaN,NaN,NaN,2.0,9.0,Top,NaN,NaN,605131.0,-5.950264,-140.490456,-7.897391,3.772000,32.321911,-8.981441,3.5,1.69,NaN,NaN,NaN,95.4,2615.0,5.7,635886.0,605131.0,571970.0,571771.0,621458.0,608369.0,621035.0,641355.0,605141.0,54.82,NaN,NaN,0.0,1.0,0.0,0.0,NaN,65.0,3.0,4-Seam Fastball,3.0,1.0,1.0,3.0,1.0,3.0,1.0,3.0,Standard,Standard
1,1,FF,2020-10-27,94.1,2.91,5.45,Julio Urias,642715.0,628711.0,NaN,called_strike,1.0,NaN,W,R,L,LAD,TB,S,NaN,NaN,0.0,1.0,2020.0,0.8,1.3,-0.55,3.03,NaN,NaN,NaN,2.0,9.0,Top,NaN,NaN,605131.0,-10.560246,-136.599519,-3.429867,11.723598,29.183810,-15.237217,3.5,1.69,NaN,NaN,NaN,93.4,2470.0,5.9,635886.0,605131.0,571970.0,571771.0,621458.0,608369.0,621035.0,641355.0,605141.0,54.59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65.0,2.0,4-Seam Fastball,3.0,1.0,1.0,3.0,1.0,3.0,1.0,3.0,Standard,Standard


These will be the selected features.

```python
features = ['index', 'pitch_type','game_date', 'release_speed', 'release_pos_x', 'release_pos_z', 
            'player_name', 'batter', 'events', 'description', 'stand', 'p_throws', 'balls', 'strikes',
            'type', 'bb_type', 'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az',
            'pfx_x', 'pfx_z', 'plate_x', 'plate_z', 'on_3b', 'on_2b', 'on_1b', 'inning', 'inning_topbot',
            'effective_speed', 'release_spin_rate', 'pitcher','fielder_2', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6',
            'fielder_7', 'fielder_8', 'fielder_9', 'at_bat_number', 'pitch_number', 'pitch_name',
            'bat_score', 'fld_score', 'post_bat_score', 'post_fld_score', 'if_fielding_alignment', 'of_fielding_alignment']
```

In [100]:
features = ['index', 'pitch_type','game_date', 'release_speed', 'release_pos_x', 'release_pos_z', 
            'player_name', 'batter', 'events', 'description', 'stand', 'p_throws', 'balls', 'strikes',
            'type', 'bb_type', 'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az',
            'pfx_x', 'pfx_z', 'plate_x', 'plate_z', 'on_3b', 'on_2b', 'on_1b', 'outs_when_up', 'inning', 'inning_topbot',
            'effective_speed', 'pitcher','fielder_2', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6',
            'fielder_7', 'fielder_8', 'fielder_9', 'at_bat_number', 'pitch_number', 'pitch_name',
            'bat_score', 'fld_score', 'post_bat_score', 'post_fld_score', 'if_fielding_alignment', 'of_fielding_alignment',
            'home_team', 'away_team']

In [101]:
mlb2020new = mlb2020[features].copy()
mlb2020new.shape

(279660, 53)

In [102]:
mlb2020new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 279660 entries, 0 to 279659
Data columns (total 53 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   index                  279660 non-null  int64  
 1   pitch_type             279089 non-null  object 
 2   game_date              279660 non-null  object 
 3   release_speed          279652 non-null  float64
 4   release_pos_x          279652 non-null  float64
 5   release_pos_z          279652 non-null  float64
 6   player_name            279660 non-null  object 
 7   batter                 279660 non-null  float64
 8   events                 70437 non-null   object 
 9   description            279660 non-null  object 
 10  stand                  279660 non-null  object 
 11  p_throws               279660 non-null  object 
 12  balls                  279660 non-null  float64
 13  strikes                279660 non-null  float64
 14  type                   279660 non-nu

# Cleaning

### Check for Nulls

---
This will be a form of cleaning. We need to know what needs to be imputed, or can be imputed, prior to EDA and if possible modeling.

In [103]:
mlb2020new.isnull().sum()
# events nulls mean same at-bat
# bb_type means not a defensive play for an out, basically no out by fielders
# release speed, pos_(x/z), v(x/y/z)0, a(x,y,z), pfx_(x/z), plate_(x/z), effective speed are all the rows, will either drop or try to get the data
# on_(3/2/1)b means if there is a runner on base using ID, NaN means empty, will fill with 0s

# treat events, bb_type, as possible targets

index                         0
pitch_type                  571
game_date                     0
release_speed                 8
release_pos_x                 8
release_pos_z                 8
player_name                   0
batter                        0
events                   209223
description                   0
stand                         0
p_throws                      0
balls                         0
strikes                       0
type                          0
bb_type                  233143
vx0                           8
vy0                           8
vz0                           8
ax                            8
ay                            8
az                            8
pfx_x                         8
pfx_z                         8
plate_x                       8
plate_z                       8
on_3b                    252778
on_2b                    226576
on_1b                    191254
outs_when_up                  0
inning                        0
inning_t

In [104]:
# Pitch Type Unknown
# Could be due to position players pitching and not having a known pitch repertoir
# Flatout just don't know the pitch name
mlb2020new[mlb2020new['pitch_type'].isnull()]
# We will fillna ('Unknown') into `pitch_type` and `pitch_name`

,index,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,events,description,stand,p_throws,balls,strikes,type,bb_type,vx0,vy0,vz0,ax,ay,az,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,effective_speed,pitcher,fielder_2,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,at_bat_number,pitch_number,pitch_name,bat_score,fld_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,home_team,away_team
36753,13255,NaN,2020-09-23,NaN,NaN,NaN,Brock Holt,666160.0,NaN,ball,L,R,3.0,2.0,B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,664068.0,2.0,9.0,Top,NaN,571788.0,543228.0,452678.0,543281.0,669738.0,671277.0,664057.0,572191.0,665742.0,74.0,6.0,NaN,12.0,1.0,12.0,1.0,NaN,NaN,WSH,PHI
36758,13322,NaN,2020-09-23,NaN,NaN,NaN,Brock Holt,666160.0,NaN,ball,L,R,0.0,0.0,B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,664068.0,2.0,9.0,Top,NaN,571788.0,543228.0,452678.0,543281.0,669738.0,671277.0,664057.0,572191.0,665742.0,74.0,1.0,NaN,12.0,1.0,12.0,1.0,Infield shift,Strategic,WSH,PHI
116769,21049,NaN,2020-09-04,96.5,2.66,5.68,Jesus Luzardo,595777.0,strikeout,called_strike,R,L,0.0,2.0,S,NaN,-11.025848,-139.957473,-4.644491,22.136325,35.480857,-15.243990,1.5,1.2,-0.28,2.83,NaN,571976.0,669134.0,1.0,5.0,Top,94.6,666200.0,669221.0,621566.0,600303.0,656305.0,605353.0,543257.0,657656.0,572039.0,32.0,3.0,NaN,0.0,0.0,0.0,0.0,Standard,Strategic,OAK,SD
116774,21155,NaN,2020-09-04,95.9,2.59,5.76,Jesus Luzardo,669134.0,NaN,ball,R,L,1.0,1.0,B,NaN,-12.910605,-138.901157,-6.263698,20.772579,33.033315,-15.377791,1.4,1.2,-1.19,2.24,NaN,571976.0,NaN,1.0,5.0,Top,94.4,666200.0,669221.0,621566.0,600303.0,656305.0,605353.0,543257.0,657656.0,572039.0,31.0,3.0,NaN,0.0,0.0,0.0,0.0,Strategic,Standard,OAK,SD
116813,21925,NaN,2020-09-04,95.1,2.48,5.74,Jesus Luzardo,543333.0,NaN,ball,L,L,0.0,0.0,B,NaN,-6.509082,-138.244707,-3.448869,19.311341,34.630885,-15.642430,1.4,1.3,1.16,3.30,NaN,NaN,NaN,1.0,4.0,Top,93.5,666200.0,669221.0,621566.0,600303.0,656305.0,605353.0,543257.0,657656.0,572039.0,23.0,1.0,NaN,0.0,0.0,0.0,0.0,Standard,Standard,OAK,SD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249677,10517,NaN,2020-07-31,NaN,NaN,NaN,Evan Marshall,643436.0,NaN,foul,R,R,1.0,1.0,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,8.0,Bot,NaN,605359.0,543510.0,547989.0,663611.0,660162.0,544725.0,650391.0,673357.0,641553.0,65.0,3.0,NaN,2.0,3.0,2.0,3.0,Infield shift,Standard,KC,CWS
249814,12159,NaN,2020-07-31,NaN,NaN,NaN,Kris Bubic,673357.0,NaN,foul,R,L,0.0,1.0,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,4.0,Top,NaN,663460.0,521692.0,643436.0,670032.0,596748.0,609275.0,460086.0,605490.0,593160.0,29.0,2.0,NaN,3.0,1.0,3.0,1.0,Standard,Standard,KC,CWS
251011,11456,NaN,2020-07-31,NaN,NaN,NaN,Taijuan Walker,657656.0,NaN,ball,R,R,0.0,0.0,B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,6.0,Top,NaN,592836.0,543592.0,657108.0,643418.0,572122.0,641487.0,543829.0,641786.0,664238.0,44.0,1.0,NaN,0.0,5.0,0.0,5.0,Standard,Standard,SEA,OAK
251013,11490,NaN,2020-07-31,NaN,NaN,NaN,Taijuan Walker,543760.0,NaN,ball,R,R,2.0,2.0,B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,6.0,Top,NaN,592836.0,543592.0,657108.0,643418.0,572122.0,641487.0,543829.0,641786.0,664238.0,43.0,6.0,NaN,0.0,5.0,0.0,5.0,Standard,Standard,SEA,OAK


In [105]:
mlb2020new['of_fielding_alignment'].value_counts(normalize=True) # Drop this column when modeling, use for EDA
# Fill with Unknown

Standard          0.925595
Strategic         0.072424
4th outfielder    0.001981
Name: of_fielding_alignment, dtype: float64

In [106]:
mlb2020new['if_fielding_alignment'].value_counts(normalize=True) # Fill with Unknown

Standard         0.561735
Infield shift    0.348100
Strategic        0.090164
Name: if_fielding_alignment, dtype: float64

### `Fillna`

---
I will fill/impute any missing values with appropriate values.

In [107]:
## Up for debate
# mlb2020new['events'].fillna('same at-bat',inplace=True)
mlb2020new['events'].value_counts(normalize=True)

field_out                     0.383875
strikeout                     0.235260
single                        0.136221
walk                          0.088831
double                        0.042052
home_run                      0.034967
force_out                     0.019493
grounded_into_double_play     0.018371
hit_by_pitch                  0.012394
field_error                   0.007709
sac_fly                       0.006034
triple                        0.003535
double_play                   0.002243
fielders_choice               0.002201
sac_bunt                      0.001831
fielders_choice_out           0.001789
caught_stealing_2b            0.001051
strikeout_double_play         0.000639
interf_def                    0.000525
pickoff_caught_stealing_2b    0.000213
pickoff_1b                    0.000170
other_out                     0.000156
pickoff_2b                    0.000099
caught_stealing_3b            0.000085
caught_stealing_home          0.000085
sac_fly_double_play      

In [108]:
mlb2020new[mlb2020new['on_1b'].notnull()]

,index,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,events,description,stand,p_throws,balls,strikes,type,bb_type,vx0,vy0,vz0,ax,ay,az,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,effective_speed,pitcher,fielder_2,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,at_bat_number,pitch_number,pitch_name,bat_score,fld_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,home_team,away_team
13,13,SI,2020-10-27,87.3,2.80,5.62,Ryan Yarbrough,571970.0,grounded_into_double_play,hit_into_play,L,L,1.0,0.0,X,ground_ball,-8.876187,-126.909224,-2.863199,13.515807,24.416540,-23.996752,1.1,0.7,0.05,2.50,NaN,NaN,608369.0,1.0,8.0,Bot,87.7,642232.0,605421.0,650490.0,664040.0,621563.0,642715.0,668227.0,595281.0,622534.0,62.0,2.0,Sinker,3.0,1.0,3.0,1.0,Infield shift,Standard,LAD,TB
14,14,CU,2020-10-27,72.7,2.94,5.18,Ryan Yarbrough,571970.0,NaN,ball,L,L,0.0,0.0,B,NaN,-6.650250,-105.570737,4.434661,-7.679582,18.495294,-31.674538,-1.2,0.2,-1.40,3.85,NaN,NaN,608369.0,1.0,8.0,Bot,72.5,642232.0,605421.0,650490.0,664040.0,621563.0,642715.0,668227.0,595281.0,622534.0,62.0,1.0,Curveball,3.0,1.0,3.0,1.0,Infield shift,Standard,LAD,TB
15,15,FF,2020-10-27,98.0,0.07,6.77,Pete Fairbanks,571771.0,strikeout,swinging_strike,R,R,1.0,2.0,S,NaN,1.363512,-142.729662,-6.710995,-1.015568,31.662677,-11.390841,-0.1,1.4,0.52,3.51,NaN,NaN,608369.0,0.0,8.0,Bot,99.2,664126.0,605421.0,650490.0,664040.0,621563.0,642715.0,668227.0,595281.0,622534.0,61.0,4.0,4-Seam Fastball,3.0,1.0,3.0,1.0,Standard,Standard,LAD,TB
16,16,SL,2020-10-27,86.8,-0.10,6.85,Pete Fairbanks,571771.0,NaN,blocked_ball,R,R,0.0,2.0,B,NaN,0.271377,-126.375895,-6.850074,2.073877,24.686078,-34.415774,0.2,-0.3,0.18,1.16,NaN,NaN,608369.0,0.0,8.0,Bot,87.6,664126.0,605421.0,650490.0,664040.0,621563.0,642715.0,668227.0,595281.0,622534.0,61.0,3.0,Slider,3.0,1.0,3.0,1.0,Standard,Standard,LAD,TB
17,17,FF,2020-10-27,96.5,-0.23,6.74,Pete Fairbanks,571771.0,NaN,foul,R,R,0.0,1.0,S,NaN,-0.394390,-140.533791,-6.632665,0.169652,30.960733,-11.833674,0.0,1.4,-0.37,3.41,NaN,NaN,608369.0,0.0,8.0,Bot,97.6,664126.0,605421.0,650490.0,664040.0,621563.0,642715.0,668227.0,595281.0,622534.0,61.0,2.0,4-Seam Fastball,3.0,1.0,3.0,1.0,Standard,Standard,LAD,TB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279647,20139,SL,2020-07-23,86.7,-3.27,5.32,Max Scherzer,650402.0,field_out,hit_into_play,R,R,2.0,2.0,X,ground_ball,8.870280,-126.008129,-4.394708,1.632054,25.946960,-30.100450,0.3,0.1,0.71,0.99,NaN,NaN,592450.0,1.0,1.0,Top,86.7,453286.0,435559.0,519346.0,516770.0,452678.0,607208.0,664057.0,645302.0,594809.0,3.0,5.0,Slider,0.0,0.0,0.0,0.0,Infield shift,Standard,WSH,NYY
279648,20141,SL,2020-07-23,87.0,-3.43,5.29,Max Scherzer,650402.0,NaN,blocked_ball,R,R,1.0,2.0,B,NaN,9.313429,-126.395529,-5.412883,0.308150,24.223676,-30.666065,0.2,0.1,0.63,0.52,NaN,NaN,592450.0,1.0,1.0,Top,87.1,453286.0,435559.0,519346.0,516770.0,452678.0,607208.0,664057.0,645302.0,594809.0,3.0,4.0,Slider,0.0,0.0,0.0,0.0,Infield shift,Standard,WSH,NYY
279649,20144,FF,2020-07-23,95.1,-3.35,5.21,Max Scherzer,650402.0,NaN,called_strike,R,R,1.0,1.0,S,NaN,12.277962,-138.031178,-4.446755,-12.511051,28.755416,-16.427966,-0.8,1.2,0.68,2.36,NaN,NaN,592450.0,1.0,1.0,Top,95.3,453286.0,435559.0,519346.0,516770.0,452678.0,607208.0,664057.0,645302.0,594809.0,3.0,3.0,4-Seam Fastball,0.0,0.0,0.0,0.0,Infield shift,Standard,WSH,NYY
279650,20145,FF,2020-07-23,95.2,-3.24,5.52,Max Scherzer,650402.0,NaN,ball,R,R,0.0,1.0,B,NaN,10.896489,-138.298960,-1.709264,-9.741723,28.758854,-15.631549,-0.6,1.3,0.43,3.81,NaN,NaN,592450.0,1.0,1.0,Top,95.2,453286.0,435559.0,519346.0,516770.0,452678.0,607208.0,664057.0,645302.0,594809.0,3.0,2.0,4-Seam Fastball,0.0,0.0,0.0,0.0,Infield shift,Standard,WSH,NYY


In [109]:
# By replacing it to 1s and 0s we can determine the basics of a pitch sequence based on if a base is loaded
# So without changing the dataframe, we create if someone is on base or not
mlb2020new['first'] = np.where(mlb2020new['on_1b'].notnull(),1,0)
mlb2020new['second'] = np.where(mlb2020new['on_2b'].notnull(),1,0)
mlb2020new['third'] = np.where(mlb2020new['on_3b'].notnull(),1,0)

# Check the value counts
pd.Series(np.where(mlb2020new['on_1b'].notnull(),1,0)).value_counts(normalize=True), \
pd.Series(np.where(mlb2020new['on_2b'].notnull(),1,0)).value_counts(normalize=True), \
pd.Series(np.where(mlb2020new['on_3b'].notnull(),1,0)).value_counts(normalize=True)

(0    0.68388
 1    0.31612
 dtype: float64,
 0    0.810184
 1    0.189816
 dtype: float64,
 0    0.903876
 1    0.096124
 dtype: float64)

In [113]:
# Drop the 8 rows of data that have nothing but NaNs
# release speed, pos_(x/z), v(x/y/z)0, a(x,y,z), pfx_(x/z), plate_(x/z), effective speed 
# are all the rows, will either drop or try to get the data
mlb2020new = mlb2020new[mlb2020new['release_speed'].notnull()]

In [142]:
# Replace CS with CU since they are both Curveballs
mlb2020new[mlb2020new['pitch_type']=='CU']['pitch_name'].unique()

array(['Curveball'], dtype=object)

In [145]:
# Put `Unknown` in pitch_name, if/of alignment 
# Put 'UN' for pitch_type = unidentified

# does not account for two seamers in pitch_name/pitch_type

# Overall I think I will drop them instead
mlb2020new = mlb2020new[mlb2020new['pitch_name'].notnull()]

In [153]:
# for infield and outfield, they are the same so dropping one drops the other as well
mlb2020new = mlb2020new[mlb2020new['of_fielding_alignment'].notnull()]

In [154]:
mlb2020new.isnull().sum()
# When it comes to modeling, I will not use events, bb_type, on_(3/2/1)b as features

index                         0
pitch_type                    0
game_date                     0
release_speed                 0
release_pos_x                 0
release_pos_z                 0
player_name                   0
batter                        0
events                   208359
description                   0
stand                         0
p_throws                      0
balls                         0
strikes                       0
type                          0
bb_type                  232188
vx0                           0
vy0                           0
vz0                           0
ax                            0
ay                            0
az                            0
pfx_x                         0
pfx_z                         0
plate_x                       0
plate_z                       0
on_3b                    251793
on_2b                    225718
on_1b                    190517
outs_when_up                  0
inning                        0
inning_t

This will be my upcoming X
```python
X = mlb2020new.drop(columns=['events', 'bb_type', 'on_3b', 'on_2b', 'on_1b'])
```

### Save Data as Cleaned Data

In [157]:
mlb2020new.to_csv('data/cleaned2020.csv', index=False)